Machine Transaltion Project 

In [3]:
print('hii')

hii


In [4]:
import collections
import numpy as np
import json

In [5]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.10.1'

In [8]:
from tensorflow.keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, GRU, LSTM, Bidirectional, Dropout, Activation, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

verify access to the GPU

In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3605074322923439580
xla_global_id: -1
]


Dataset

for our machine translation project we opt for a dataset featuring a limited vocabulary, specifically designed to facilitate a more manageable and efficient training process. Unlike the extensive WMT datasets, our chosen dataset ensures a quicker training time and demands fewer computational resources. This strategic decision aims to balance the learning experience while still achieving meaningful results within practical time constraints. 

Load Dataset

In [10]:
def load_data(path) :
    input_file = path
    with open(input_file, "r") as f :
        data = f.read()
    return data.split('\n')

english_sentences = load_data('data/english.txt')
french_sentences = load_data('data/french.txt')

In [11]:
english_sentences[1]

'the united states is usually chilly during july , and it is usually freezing in november .'

In [12]:
french_sentences[1]

'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .'

sample data 

In [13]:
english_sentences[:5]

['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .']

By examining the sentences, it's apparent that they undergone preprocessing, punctuation has been delimited with spaces, and all the text has been converted to lowercase. This preprocessing serves a crucial purpose in text preparation. Firstly, delimiting punctuation with spaces ensures that each punctuation mark is treated as a seperate token, aiding the model in understanding sentence structure. Secondly, converting the entire text to lowercase standardizes the input, preventing the model from distinguishing between words solely based on their casing. This uniformity facilitates more effective training and generalization, enhancing the model's ability to grasp patterns and generate accurate translations. 

Structure of Dataset 

In [14]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 most common words in english  dataset: ')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')

print()
print('{} french words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 most common words in french dataset: ')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')


1823250 English words.
227 unique English words.
10 most common words in english  dataset: 
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 french words.
355 unique French words.
10 most common words in french dataset: 
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


Preprocess

1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

In [15]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .' 
]

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, tokn_sent) in enumerate(zip(text_sentences, text_tokenized)) :
    print('Sequence {} in x'.format(sample_i + 1))
    print(' Input: {}'.format(sent))
    print(' Output: {}'.format(tokn_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
 Input: The quick brown fox jumps over the lazy dog .
 Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
 Input: By Jove , my quick study of lexicography won a prize .
 Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
 Input: This is a short sentence .
 Output: [18, 19, 3, 20, 21]


In [16]:
def pad(x, length= None) :
    if length is None :
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen= length, padding= 'post')

test_pad = pad(text_tokenized) 
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)) :
    print('Sequence {} in x'.format(sample_i + 1))
    print(' Input: {}'.format(np.array(token_sent)))
    print(' Output: {}'.format(pad_sent))

Sequence 1 in x
 Input: [1 2 4 5 6 7 1 8 9]
 Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
 Input: [10 11 12  2 13 14 15 16  3 17]
 Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
 Input: [18 19  3 20 21]
 Output: [18 19  3 20 21  0  0  0  0  0]


In [17]:
def preprocess(x,y) :
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x) 
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index) 
french_vocab_size = len(french_tokenizer.word_index) 

print('Data Preprocessed') 
print("max english sentence length: ", max_english_sequence_length)
print("max french sentence length: ", max_french_sequence_length)
print("english vocabulary size: ", english_vocab_size) 
print("french vocabulary size: ", french_vocab_size)


Data Preprocessed
max english sentence length:  15
max french sentence length:  21
english vocabulary size:  199
french vocabulary size:  344


Models

* Model 1 is a simple RNN
* Model 2 is a Bidirectional RNN
* Model 3 is a Embedding RNN

Ids Back to Text

The neural network will be translating the input to words ids, which isn't thefinal form we want. We want the French translation. The function logits_to_text will bridge the gap between the logits from the neural network to the French translation. You'll be using this function to better understand the output of the neural network.  

In [18]:
def logits_to_text(logits, tokenizer) :
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])


Model 1 : RNN

In [19]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size) :
    # hyperparameters
    learning_rate = 0.005

    # build the layers
    model = Sequential()
    model.add(GRU(256, input_shape= input_shape[1:], return_sequences= True))
    model.add(TimeDistributed(Dense(1024, activation = 'relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation= 'softmax')))

    # compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy']
                  )
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# Ensure labels are integers
if preproc_french_sentences.ndim == 3:
    preproc_french_sentences = np.argmax(preproc_french_sentences, axis=-1)

# Debugging shapes
print("Input shape (tmp_x):", tmp_x.shape)
print("Label shape (preproc_french_sentences):", preproc_french_sentences.shape)

# Train the model
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size
)

simple_rnn_model.fit(
    tmp_x,
    preproc_french_sentences,
    batch_size=1024,
    epochs=10,
    validation_split=0.2
)

Input shape (tmp_x): (137861, 21, 1)
Label shape (preproc_french_sentences): (137861, 21)
Epoch 1/10
108/108 [==============================] - 173s 2s/step - loss: 0.0735 - accuracy: 0.9907 - val_loss: 3.4809e-05 - val_accuracy: 1.0000
Epoch 2/10
108/108 [==============================] - 152s 1s/step - loss: 3.4808e-05 - accuracy: 1.0000 - val_loss: 3.4809e-05 - val_accuracy: 1.0000
Epoch 3/10
108/108 [==============================] - 108s 1s/step - loss: 3.4808e-05 - accuracy: 1.0000 - val_loss: 3.4809e-05 - val_accuracy: 1.0000
Epoch 4/10
108/108 [==============================] - 110s 1s/step - loss: 3.4808e-05 - accuracy: 1.0000 - val_loss: 3.4809e-05 - val_accuracy: 1.0000
Epoch 5/10
108/108 [==============================] - 110s 1s/step - loss: 3.4808e-05 - accuracy: 1.0000 - val_loss: 3.4809e-05 - val_accuracy: 1.0000
Epoch 6/10
108/108 [==============================] - 109s 1s/step - loss: 3.4808e-05 - accuracy: 1.0000 - val_loss: 3.4809e-05 - val_accuracy: 1.0000
Epoch 7/

In [20]:

print("\nCorrect translation:")
print(french_sentences[4:5])

print("\noriginal text:")
print(english_sentences[4:5])



Correct translation:
['votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .']

original text:
['your least liked fruit is the grape , but my least liked is the apple .']


Model 2 : Bidirectional RNNs

In [30]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    # Hyperparameters
    learning_rate = 0.005

    # Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, input_shape=input_shape[1:], return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(
        loss=sparse_categorical_crossentropy,
        optimizer=Adam(learning_rate),
        metrics=['accuracy']
    )
    return model

# Ensure tmp_x shape is correct
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))  # Reshape to match (samples, sequence_length, features)

# Ensure preproc_french_sentences shape matches the model's expected target
preproc_french_sentences = preproc_french_sentences.reshape((-1, max_french_sequence_length, 1))

# Train the model
bd_rnn_model = bd_model(
    tmp_x.shape, 
    max_french_sequence_length, 
    english_vocab_size, 
    french_vocab_size
)

# Explicitly build the model
bd_rnn_model.build(input_shape=tmp_x.shape)

# Print the model summary
print(bd_rnn_model.summary())





Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (137861, 21, 256)        100608    
 nal)                                                            
                                                                 
 time_distributed_4 (TimeDis  (137861, 21, 1024)       263168    
 tributed)                                                       
                                                                 
 dropout_2 (Dropout)         (137861, 21, 1024)        0         
                                                                 
 time_distributed_5 (TimeDis  (137861, 21, 344)        352600    
 tributed)                                                       
                                                                 
Total params: 716,376
Trainable params: 716,376
Non-trainable params: 0
________________________________________________

In [ ]:
# Train the model
bd_rnn_model.fit(
    tmp_x,
    preproc_french_sentences,
    batch_size=1024,
    epochs=10,
    validation_split=0.2
)

In [22]:
tmp_x.shape

(137861, 21, 1)

In [23]:
preproc_french_sentences.shape

(137861, 21, 1)

In [37]:
 # serialize english tokenizer to json
with open('english_tokenizer.json', 'w', encoding= 'utf8') as f :
    f.write(json.dumps(english_tokenizer.to_json(), ensure_ascii= False))

# serialize french tokenizer to json 
with open('french_tokenizer.json', 'w', encoding='utf8') as f :
    f.write(json.dumps(french_tokenizer.to_json(), ensure_ascii= False))

# save max lengths 
max_french_sequence_length_json = max_french_sequence_length
with open('sequence_length.json', 'w', encoding= 'utf8') as f :
    f.write(json.dumps(max_french_sequence_length_json, ensure_ascii=False))

    
